<a href="https://colab.research.google.com/github/D4n1elR0drigues/TCC/blob/main/%5BMain_Code%5D_Trading_Robot_4_0_Deep_Q_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# EXECUTAR NA PRIMEIRA EXECUÇÃO!

In [1]:
# Imports

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

import pandas
from pandas_datareader import data as pdr
import yfinance as yfin
import datetime

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

from google.colab import drive
import shutil


# Monte o Google Drive no ambiente do Colab
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# Defining our Deep Q-Learning Trader

class AI_Trader():

# -----------------------------------------------------------------------

  # CONSTRUTOR

  def __init__(self, state_size, action_space=3, model_name="AITrader"):

    self.state_size = state_size # Tamanho da entrada da rede neural
    self.action_space = action_space # Espaço de ação será 3, Comprar, Vender, Sem Ação (Tamanho da saída da rede neural)
    self.memory = deque(maxlen=2000) # Memória com 2000 posições. A função Deque permite adicionar elementos ao final, enquanto remove elementos do início.
    self.inventory = [] # Terá as comprar que já fizemos
    self.model_name = model_name # Nome do modelo para o Keras

    self.gamma = 0.95 # Parâmetro que ajudará a maximizar a recompensa
    self.epsilon = 1.0 # Taxa de aleatoriedade para atitudes ganacioas do algorítimo.
    self.epsilon_final = 0.01 # Taxa final reduzida
    self.epsilon_decay = 0.995 # Velocidade de decaimento da taxa

    self.model = self.model_builder() # Inicializa um modelo e de rede neural e salva na classe

# -----------------------------------------------------------------------

  # DEFININDO A REDE NEURAL

  def model_builder(self):

    model = tf.keras.models.Sequential()
    model.add(layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    model.add(layers.Dense(units=64, activation='relu'))
    model.add(layers.Dense(units=128, activation='relu'))
    model.add(layers.Dense(units=self.action_space, activation='linear')) # De maneira geral, teremos 3 saída na rede geral (número de espaços de ação)


    model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001)); # Compilamos o modelo

    return model # Retornamos o modelo pela função.

# -----------------------------------------------------------------------

  # FUNÇÃO DE TRADE
  # Usa o Epsilon e um número aleatório para definir se usará um dado aleatório ou a previsão da rede.

  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space) # Retonar uma resposta aleatória

    actions = self.model.predict(state)
    return np.argmax(actions[0]) # Retorna o index da maior resposta da rede

# -----------------------------------------------------------------------

  # LOTE DE TREINAMENTO

  # Definindo o modelo para treinamento do lote

  def batch_train(self, batch_size): # Função que tem o tamanho do lote como argumento

    batch = [] # Iremos usar a memória como lote, por isso iniciamos com uma lista vazia

    # Iteramos sobre a memória, adicionando seus elementos ao lote batch
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])

    # Agora temos um lote de dados e devemos iterar sobre cada estado, recompensa,
    # proximo_estado e conclusão do lote e treinar o modelo com isso.
    for state, action, reward, next_state, done in batch:
      reward = reward

      # Se não estivermos no último agente da memória, então calculamos a
      # recompensa descontando a recompensa total da recompensa atual.
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])

      # Fazemos uma previsão e alocamos à varivel target
      target = self.model.predict(state)
      target[0][action] = reward

      # Treinamos o modelo com o estado, usando a previsão como resultado esperado.
      self.model.fit(state, target, epochs=1, verbose=0)

    # Por fim decrementamos o epsilon a fim de gradativamente diminuir tentativas ganaciosas.
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

# -----------------------------------------------------------------------


# -----------------------------------------------------------------------


# -----------------------------------------------------------------------


# -----------------------------------------------------------------------



In [3]:
# Stock Market Data Preprocessing

# Definiremos algumas funções auxiliares

# Sigmoid
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

# Função para formatar texto
def stock_price_format(n):
  if n < 0:
    return "- # {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

# Busca dados no Yahoo Finance
# Formato data = "yyyy-mm-dd"
def dataset_loader(stock_name, initial_date, final_date):

  yfin.pdr_override()

  dataset = pdr.get_data_yahoo(stock_name, start=initial_date, end=final_date)

  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[1]).split()[0]

  close = dataset['Close']

  return close

In [4]:
# State Creator


def state_creator(data, timestep, window_size):

  # O index incial (starting_id) será o timestep (passos/dias que já foram dados)
  # menos o tamanho da janela, que serão os dias olhados para trás.
  starting_id = timestep - window_size + 1


  # Lógica para preencher os dados vindos da tabela Data, no array windowed_data

  if starting_id >= 0: # No geral este será a condição sempre executada
    windowed_data = data[starting_id: timestep + 1]

  else: # Condição executada apenas nos primeiros valores
    windowed_data =- starting_id * [data[0]] + list(data[0:timestep + 1])

  state = [] # Criação de uma array para retorno, com o estado.

  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))

  return np.array([state])

In [32]:
# Loading a Dataset

# CONFIGURAÇÕES DE IMPORTAÇÃO DE DADOS

# NOME DA AÇÃO
STOCK_NAME = "PETR4.SA"

# DATA INCIAL
INITIAL_DATE = "2023-05-01"

# DATA FINAL
today = datetime.date.today()
FINAL_DATE = today.strftime("%Y-%m-%d") # Escolhe a data final como hoje

data = dataset_loader(STOCK_NAME, INITIAL_DATE, FINAL_DATE);

data

[*********************100%%**********************]  1 of 1 completed


Date
2023-05-02    22.740000
2023-05-03    22.660000
2023-05-04    23.020000
2023-05-05    24.000000
2023-05-08    24.510000
                ...    
2023-10-18    38.520000
2023-10-19    38.340000
2023-10-20    37.849998
2023-10-23    35.349998
2023-10-24    35.880001
Name: Close, Length: 123, dtype: float64

In [33]:
# Training the Q-Learning Trading Agent

window_size = 10
episodes = 2

batch_size = 32
data_samples = len(data) - 1

trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_100 (Dense)           (None, 32)                352       
                                                                 
 dense_101 (Dense)           (None, 64)                2112      
                                                                 
 dense_102 (Dense)           (None, 128)               8320      
                                                                 
 dense_103 (Dense)           (None, 3)                 387       
                                                                 
Total params: 11171 (43.64 KB)
Trainable params: 11171 (43.64 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
# Vamos iterar sobre todos episódios

for episode in range(1, episodes + 1):

  print("Episode: {}/{}".format(episode, episodes))

  # Criamos o primeiro estado.
  state = state_creator(data, 0, window_size + 1)

  # Inicializamos algumas variáveis
  total_profit = 0
  trader.inventory = []

  # O Loop de treinamento que será executado durante uma época inteira
  for t in tqdm(range(data_samples)):

    # O IA executa a função trade, que responderá com a ação que deve ser tomada
    action = trader.trade(state)

    # Já definimos o próximo estado
    # Note que o definimos com t+1, pois estamos considerando o próximo
    # valor da ação no index da tabela de dados.
    next_state = state_creator(data, t+1, window_size + 1)

    # Sem recompensas até agora
    reward = 0

    # Sem ação
    if action == 0:
      # Apenas um print e Recompensa = 0
      print(" - Sem ação | Total de papeis no portfolio = ", len(trader.inventory))

    # Compra
    if action == 1:
      # Recompensa = 0

      # Adicionamos a ação comprada na array de portfolio
      trader.inventory.append(data[t])

      print(" - AI Trader Comprou: ", stock_price_format(data[t]))

    # Venda (Deve possuir ações no portfolio)
    elif action == 2 and len(trader.inventory) > 0:

      # Remove última ação do portfólio e a retorna
      buy_price = trader.inventory.pop(0)

      # Recompensa = lucro ou 0 se houve prejuízo.
      reward = max(data[t] - buy_price, 0)

      total_profit += data[t] - buy_price # Soma ao lucro/prejuízo total

      print(" - AI Trader Vendeu: ", stock_price_format(data[t]), " - Lucro: " + stock_price_format(data[t] - buy_price) )


    # Verifica se estamos no final de uma época
    if t == data_samples - 1:
      done = True
    else:
      done = False


    # Salvamos os dados na memória, na mesma ordem que na função BATCH_TRAIN
    trader.memory.append((state, action, reward, next_state, done))

    # Definimos que o estado atual é o próximo estado calculado anteriormente
    state = next_state

    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")


    # Se o tamanho da memória for maior que o tamanho do lote que definimos
    # Então vamos treinar a rede, passando o tamanho do lote como argumento
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  # A Cada 10 episódios treinados, salvamos a rede
  if episode % 2 == 0:
      model_name = "ai_trader_{}.h5".format(episode)
      trader.model.save(model_name)

      # Enviar o arquivo para o Google Drive
      source_path = model_name
      destination_path = '/content/drive/My Drive/Arquivos H5/' + model_name

      shutil.copy(source_path, destination_path)

      print('Arquivo enviado para o Google Drive com sucesso!')

Episode: 1/2


  0%|          | 0/122 [00:00<?, ?it/s]

 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - AI Trader Comprou:  $ 25.660000
 - Sem ação | Total de papeis no portfolio =  1
 - AI Trader Comprou:  $ 25.660000
 - AI Trader Vendeu:  $ 25.809999  - Lucro: $ 0.150000
 - AI Trader Vendeu:  $ 25.770000  - Lucro: $ 0.110001
 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - AI Trader Comprou:  $ 26.389999
 - Sem ação | Total de papeis no portfolio =  1
 - AI Trader Vendeu:  $ 26.959999  - Lucro: $ 0.570000
 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - AI Trader Comprou:  $ 30.280001
 - AI Trader Vendeu:  $ 30.809999  - Lucro: $ 0.529999
 - AI Trader Comprou:  $ 30.100000
 - AI Trader Comprou:  $ 29.389999
 - Sem ação | Total de papeis no portfolio

 27%|██▋       | 33/122 [00:08<00:22,  3.94it/s]

 - Sem ação | Total de papeis no portfolio =  2
1/1 [==============================] - 0s 19ms/step


...

 99%|█████████▉| 121/122 [13:14<00:10, 10.40s/it]

1/1 [==============================] - 2s 2s/step
 - AI Trader Comprou:  $ 35.349998
########################
TOTAL PROFIT: 0.5499954223632812
########################
1/1 [==============================] - 0s 17ms/step


100%|██████████| 122/122 [14:29<00:00,  7.13s/it]

Arquivo enviado para o Google Drive com sucesso!



/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [43]:
# Defina uma função para carregar e executar o modelo treinado
def run_trained_model(model_path, data, window_size):
    trader = AI_Trader(window_size)
    trader.model = tf.keras.models.load_model(model_path)

    state = state_creator(data, 0, window_size + 1)
    total_profit = 0
    trader.inventory.clear()
    trader.inventory = []
    print("quantidade 555 ", len(trader.inventory))

    for t in tqdm(range(len(data) - 1)):
        action = trader.trade(state)
        next_state = state_creator(data, t + 1, window_size + 1)

        # Sem ação
        if action == 0:
          # Apenas um print e Recompensa = 0
          print(" - Sem ação | Total de papeis no portfolio = ", len(trader.inventory))

        # Compra
        if action == 1:
          # Recompensa = 0

          # Adicionamos a ação comprada na array de portfolio
          trader.inventory.append(data[t])

          print(" - AI Trader Comprou: ", stock_price_format(data[t]))

        # Venda (Deve possuir ações no portfolio)
        elif action == 2 and len(trader.inventory) > 0:

          # Remove última ação do portfólio e a retorna
          buy_price = trader.inventory.pop(0)

          # Recompensa = lucro ou 0 se houve prejuízo.
          reward = max(data[t] - buy_price, 0)

          total_profit += data[t] - buy_price # Soma ao lucro/prejuízo total

          print(" - AI Trader Vendeu: ", stock_price_format(data[t]), " - Lucro: " + stock_price_format(data[t] - buy_price) )
        state = next_state

    return total_profit

# Caminho do modelo no seu Google Drive
model_path = "/content/drive/My Drive/Arquivos H5/ai_trader_2.h5"  # Substitua pelo caminho correto

# Exemplo de uso
new_data = dataset_loader(STOCK_NAME, "2023-01-01", "2023-02-20")  # Carrega novos dados

# Executa o modelo treinado nos novos dados
profit = run_trained_model(model_path, new_data, window_size)

print(f"Lucro total: {profit}")


[*********************100%%**********************]  1 of 1 completed
quantidade 555  0


  0%|          | 0/34 [00:00<?, ?it/s]

 - Sem ação | Total de papeis no portfolio =  0
 - Sem ação | Total de papeis no portfolio =  0
 - AI Trader Comprou:  $ 23.740000
 - AI Trader Comprou:  $ 23.870001
 - Sem ação | Total de papeis no portfolio =  2
 - AI Trader Vendeu:  $ 24.280001  - Lucro: $ 0.540001
 - AI Trader Vendeu:  $ 24.629999  - Lucro: $ 0.759998
 - AI Trader Comprou:  $ 25.520000
 - Sem ação | Total de papeis no portfolio =  1
 - AI Trader Vendeu:  $ 25.889999  - Lucro: $ 0.369999
 - Sem ação | Total de papeis no portfolio =  0
 - AI Trader Comprou:  $ 26.600000
 - AI Trader Comprou:  

100%|██████████| 34/34 [00:00<00:00, 1993.60it/s]

$ 26.940001
 - AI Trader Comprou:  $ 26.250000
 - AI Trader Comprou:  $ 25.620001
 - AI Trader Vendeu:  $ 25.750000  - Lucro: - # 0.850000
 - AI Trader Vendeu:  $ 26.070000  - Lucro: - # 0.870001
 - AI Trader Comprou:  $ 25.709999
 - AI Trader Vendeu:  $ 24.520000  - Lucro: - # 1.730000
 - AI Trader Comprou:  $ 24.790001
 - AI Trader Comprou:  $ 25.780001
 - AI Trader Comprou:  $ 25.620001
 - AI Trader Vendeu:  $ 26.049999  - Lucro: $ 0.429998
 - AI Trader Vendeu:  $ 25.930000  - Lucro: $ 0.220001
 - AI Trader Vendeu:  $ 26.719999  - Lucro: $ 1.929998
 - AI Trader Comprou:  $ 26.799999
 - AI Trader Vendeu:  $ 26.700001  - Lucro: $ 0.920000
 - AI Trader Comprou:  $ 26.740000
 - AI Trader Comprou:  $ 26.850000
Lucro total: 1.7199954986572266
